In [1]:
from packages import *

In [ ]:
data_path = "../DATA/A-2/"
output_path = "outputs/A-2/"
filename = lambda gain: f"{data_path}A-1_gain{gain}_100mV.csv"
getrawdata = lambda gain: pd.read_csv(filename(gain))
COLUMN_NAMES = ['frequency [Hz]', 'output p-p [V]', 'input p-p [V]', 'Gain (o/i)']
MILIVOLT_NAMES = ['output p-p [mV]', 'input p-p [mV]']

def get_data(gain):
  # freq, output, input, gain = getrawdata(gain)
  data = getrawdata(gain)
  data.drop(columns=["Gain (o/i)"], inplace=True)

  freq = data['frequency [Hz]']

  if MILIVOLT_NAMES[0] in data.columns:
    output = data[MILIVOLT_NAMES[0]] / 1000
    input = data[MILIVOLT_NAMES[1]] / 1000
  
  else:
    output = data[COLUMN_NAMES[1]]
    input = data[COLUMN_NAMES[2]]

  return freq, output, input

In [ ]:
xrange = (1e2, 1e6)
yrange = (-20, 20)
save = False

xmin, xmax = xrange; ymin, ymax = yrange

plt.errorbar(x, y[:, 0], yerr=y[:, 1], fmt='o', ms=5, 
            color='black', label=f'gain: {np.power(10, gain_true/10):.1f}',
            capsize=5, capthick=1)

plt.plot([xmin, xmax], [gain_true, gain_true], '--', label='True gain', color='black')
plt.plot([x_intercept, x_intersection], [y_intercept, y_intersection], 
        'r--', label='Linear fit')
plt.plot([x_goal, x_goal], [y_intercept, y_goal], 'g--', label='3 dB frequency')
xpoint = lambda ratio: 10 ** (np.log10(xmin) * (1 - ratio) + np.log10(xmax) * ratio)
ypoint = lambda ratio: ymin * (1 - ratio) + ymax * ratio
plt.text(xpoint(0.05), ypoint(0.9), f'$f = {x_goal/1000:.1f}$ kHz')
plt.text(xpoint(0.05), ypoint(0.85), f'$R^2 = {R2:.3f}$')

plt.legend(loc='lower left')
plt.xscale("log")
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Gain [dB]")
plt.title(f"Gain vs Frequency for Gain {123:.0f}")
if save: plt.savefig(f"{output_path}gain{123:.0f}.png")
plt.show()